# MyoPS 2020 Challenge


In [1]:
!pip install fastai2
!pip install git+https://github.com/Project-MONAI/MONAI#egg=MONAI

     |████████████████████████████████| 194kB 7.4MB/s 
  Cloning https://github.com/Project-MONAI/MONAI to /tmp/pip-install-zwts9r07/MONAI
  Running command git clone -q https://github.com/Project-MONAI/MONAI /tmp/pip-install-zwts9r07/MONAI
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for MONAI: filename=monai-0.2.0+44.gd7f3b98-cp36-none-any.whl size=224620 sha256=4d214fc5e3668d3929ec62efe5c0e3803bec0563316f05ab0f12ece22a416b5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-mj18crbw/wheels/32/e1/f2/9985050c97af405af0c840d4b9fc44324fdf0c92fe03de697f
Successfully built MONAI


In [2]:
from fastai2.vision.all import *
from fastai2.vision.models import resnet34
from monai.losses import FocalLoss
import gc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
path = "/content/drive/My Drive/miccai2020/myops/png"

In [5]:
np.random.seed(42)
ids = np.arange(101,126)
np.random.shuffle(ids)
ids = np.reshape(ids,(-1,5))
ids

array([[109, 117, 101, 124, 112],
       [110, 114, 102, 123, 106],
       [103, 113, 116, 104, 105],
       [121, 118, 122, 119, 125],
       [108, 111, 115, 120, 107]])

In [6]:
class AddMaskCodeMapping(Transform):
    "Add mapping of pixel value to class for a `TensorMask`"
    def __init__(self, mapping, codes=None):
        #print("init")
        self.mapping = mapping
        self.codes = codes
        if codes is not None: self.vocab,self.c = codes,len(codes)

    def encodes(self, o:PILMask):
        #print("encodes")
        mo = ToTensor()(o)
        mo = mo.to(dtype=torch.long)
        mo = self.mapping.index_select(0,mo.flatten()).reshape(*mo.shape)
        mo = PILMask.create(mo.to(dtype=torch.uint8))
        return mo
    
    def decodes(self, o:TensorMask):
        # decoding of inputs works out of the box, but get_preds are not properly decoded
        if len(o.shape) > 2:
            o = o.argmax(dim=0)
        if self.codes is not None: o._meta = {'codes': self.codes}
        return o

In [7]:
def MappedMaskBlock(mapping,codes=None):
    "A `TransformBlock` for segmentation masks, with mapping of pixel values to classes, potentially with `codes`"
    return TransformBlock(type_tfms=PILMask.create, item_tfms=AddMaskCodeMapping(mapping=mapping,codes=codes), batch_tfms=IntToFloatTensor)

In [8]:
def getMappedMaskBlock(predefined_mapping_name):
    predefined_mappings = {
        'full': ([0,1,2,3,4,5],['bg','lv','my','rv','ed','sc']),
        'edOnly': ([0,0,0,0,1,0],['bg','ed']),
        'edScCombined': ([0,0,0,0,1,1],['bg','edSc']),
        'scOnly': ([0,0,0,0,0,1],['bg','sc']),
        'edScOnly': ([0,0,0,0,1,2],['bg','ed','sc']),
    }
    mapping,codes = predefined_mappings[predefined_mapping_name]
    return MappedMaskBlock(mapping = torch.LongTensor(mapping), codes=codes)

In [9]:
def getMyopsDls(val_ids, mapping_name="full", images="images"):
    mmb = getMappedMaskBlock(mapping_name)
    myopsData = DataBlock(blocks=(ImageBlock, mmb),#['bg','lv','my','rv','ed','sc'])),
        get_items=get_image_files,
        splitter=FuncSplitter(lambda o: int(o.name.split("-")[0]) in val_ids),
        get_y=lambda o: str(o).replace(images,"masks"),
        item_tfms=CropPad(256),
        batch_tfms=aug_transforms(max_rotate=90,pad_mode="zeros"))
    dls = myopsData.dataloaders(f'{path}/{images}',num_workers=4,batch_size=12)
    dls[1].bs = 12
    return dls

In [10]:
def multi_dice(input:Tensor, targs:Tensor, class_id=0, inverse=False)->Tensor:
    n = targs.shape[0]
    input = input.argmax(dim=1).view(n,-1)
    # replace all with class_id with 1 all else with 0 to have binary case
    output = (input == class_id).float()
    # same for targs
    targs = (targs.view(n,-1) == class_id).float()
    if inverse:
        output = 1 - output
        targs = 1 - targs
    intersect = (output * targs).sum(dim=1).float()
    union = (output+targs).sum(dim=1).float()
    res = 2. * intersect / union
    res[torch.isnan(res)] = 1
    return res.mean()

def diceFG(input, targs): return multi_dice(input,targs,class_id=1)
def diceLV(input, targs): return multi_dice(input,targs,class_id=1)
def diceMY(input, targs): return multi_dice(input,targs,class_id=2)
def diceRV(input, targs): return multi_dice(input,targs,class_id=3)
def diceEd(input, targs): return multi_dice(input,targs,class_id=4)
def diceSc(input, targs): return multi_dice(input,targs,class_id=5)
dices = [diceLV,diceMY,diceRV,diceEd,diceSc]

In [11]:
def myFocal(weights=[.2,.8]):
    monaiFocal = FocalLoss(weight=torch.Tensor(weights),reduction='mean')
    return lambda input,target: monaiFocal(input, target.unsqueeze(1))

# Generic function

In [12]:
def standard_training(
    mapping="full",
    images="images",
    cleanup=True,
    metrics=dices,
    weights=[1.0/6,1.0/6,1.0/6,1.0/6,1.0/6,1.0/6],
    cv=0,
    name="standard_training",
    use_focal_loss=True,
    path='/content/drive/My Drive/miccai2020/myops',
    save=False
    ):
    my_loss = None
    if(use_focal_loss):
        my_loss = myFocal(weights)
    learn = unet_learner(
        getMyopsDls(ids[cv], mapping, images),
        resnet34,
        path=path,
        loss_func=my_loss,
        metrics=[foreground_acc, *metrics],
        cbs=[CSVLogger(f'{path}/logs/{name}.csv',append=True)]
    )
    learn.fine_tune(20, freeze_epochs=10, base_lr=1e-3)
    if save:
      learn.save(name)
    if not cleanup:
        return learn
    else:
        del learn
        gc.collect()
        torch.cuda.empty_cache()

In [13]:
for cv in range(4,5):
  if cv > 4:
    standard_training(name=f"multi_ce_cv{cv}", cv=cv, use_focal_loss=False)
    standard_training(name=f"multi_balanced_cv{cv}", cv=cv)
    standard_training(name=f"multi_pathoMyo.2_cv{cv}", cv=cv, weights=[.4/3,.4/3,.2,.4/3,.2,.2])
    standard_training(name=f"multi_pathoMyo.3_cv{cv}", cv=cv, weights=[.1/3,.1/3,.3,.1/3,.3,.3])
    standard_training(name=f"multi_patho.2_cv{cv}", cv=cv, weights=[.6/4,.6/4,.6/4,.6/4,.2,.2])
    standard_training(name=f"multi_patho.35_cv{cv}", cv=cv, weights=[.3/4,.3/4,.3/4,.3/4,.35,.35])
    standard_training(name=f"multi_patho.49_cv{cv}", cv=cv, weights=[.02/4,.02/4,.02/4,.02/4,.49,.49])
  standard_training(name=f"multi_scar.2_cv{cv}", cv=cv, weights=[.8/5,.8/5,.8/5,.8/5,.8/5,.2])
  standard_training(name=f"multi_scar.4_cv{cv}", cv=cv, weights=[.6/5,.6/5,.6/5,.6/5,.6/5,.4])
  standard_training(name=f"multi_scar.6_cv{cv}", cv=cv, weights=[.4/5,.4/5,.4/5,.4/5,.4/5,.6])
  standard_training(name=f"multi_scar.8_cv{cv}", cv=cv, weights=[.2/5,.2/5,.2/5,.2/5,.2/5,.8])
  standard_training(name=f"multi_scar.99_cv{cv}", cv=cv, weights=[.01/5,.01/5,.01/5,.01/5,.01/5,.99])
  standard_training(name=f"multi_edema.2_cv{cv}", cv=cv, weights=[.8/5,.8/5,.8/5,.8/5,.2,.8/5])
  standard_training(name=f"multi_edema.4_cv{cv}", cv=cv, weights=[.6/5,.6/5,.6/5,.6/5,.4,.6/5])
  standard_training(name=f"multi_edema.6_cv{cv}", cv=cv, weights=[.4/5,.4/5,.4/5,.4/5,.6,.4/5])
  standard_training(name=f"multi_edema.8_cv{cv}", cv=cv, weights=[.2/5,.2/5,.2/5,.2/5,.8,.2/5])
  standard_training(name=f"multi_edema.99_cv{cv}", cv=cv, weights=[.01/5,.01/5,.01/5,.01/5,.99,.01/5])
  standard_training(name=f"lge_scarOnly.5_cv{cv}", cv=cv, mapping="scOnly", images="LGE", metrics=[diceFG], weights=[0.5,0.5])
  standard_training(name=f"lge_scarOnly.8_cv{cv}", cv=cv, mapping="scOnly", images="LGE", metrics=[diceFG], weights=[0.2,0.8])
  standard_training(name=f"t2_edemaOnly.5_cv{cv}", cv=cv, mapping="edOnly", images="T2", metrics=[diceFG], weights=[0.5,0.5])
  standard_training(name=f"t2_edemaOnly.8_cv{cv}", cv=cv, mapping="edOnly", images="T2", metrics=[diceFG], weights=[0.2,0.8])

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,foreground_acc,diceLV,diceMY,diceRV,diceEd,diceSc,time
0,0.063313,0.057593,0.002042,0.002689,0.000000,0.060807,0.140351,0.052632,02:07
1,0.048625,0.043124,0.319623,0.539517,0.220321,0.388987,0.157895,0.052632,00:10
2,0.038372,0.026262,0.586058,0.741832,0.487462,0.589660,0.027665,0.052632,00:10
3,0.030281,0.029228,0.401455,0.753747,0.487818,0.304443,0.062980,0.069352,00:10
4,0.025335,0.025358,0.441393,0.765596,0.494008,0.355653,0.055448,0.116592,00:10
5,0.022107,0.021860,0.555088,0.730975,0.559875,0.583263,0.077470,0.097110,00:10
6,0.020563,0.032686,0.438393,0.714350,0.437517,0.399170,0.140644,0.092444,00:10
7,0.018231,0.018671,0.643227,0.805015,0.610738,0.659810,0.064350,0.131180,00:10
8,0.015985,0.017343,0.690190,0.821023,0.616496,0.677457,0.074425,0.164074,00:10
9,0.014506,0.015578,0.699874,0.813443,0.625298,0.700240,0.104452,0.254156,00:10


epoch,train_loss,valid_loss,foreground_acc,diceLV,diceMY,diceRV,diceEd,diceSc,time
0,0.010456,0.015689,0.707600,0.824814,0.631757,0.687743,0.108087,0.191417,00:11
1,0.009883,0.016188,0.670742,0.831436,0.621343,0.761642,0.093976,0.179201,00:11
2,0.009767,0.015367,0.646588,0.830889,0.608045,0.725271,0.148897,0.308870,00:11
3,0.009477,0.016802,0.695910,0.814186,0.597801,0.693658,0.135472,0.221546,00:11
4,0.009842,0.014740,0.715043,0.818986,0.645299,0.691308,0.094666,0.285850,00:11
5,0.009473,0.016580,0.675931,0.845435,0.636423,0.690532,0.158344,0.152611,00:11
6,0.009344,0.014203,0.743900,0.842892,0.618837,0.715240,0.162680,0.371221,00:11
7,0.009378,0.015190,0.677506,0.844775,0.657834,0.754862,0.114309,0.248989,00:11
8,0.009148,0.013613,0.725408,0.835828,0.664955,0.750510,0.179798,0.325857,00:11
9,0.008798,0.016831,0.681001,0.845316,0.618162,0.798906,0.181178,0.233894,00:11


epoch,train_loss,valid_loss,foreground_acc,diceLV,diceMY,diceRV,diceEd,diceSc,time
0,0.088105,0.063652,0.000000,0.000000,0.000000,0.087719,0.105263,0.000000,00:10
1,0.059724,0.042285,0.133540,0.490983,0.000000,0.022319,0.157895,0.000000,00:10
2,0.043892,0.033494,0.334441,0.667527,0.321547,0.258749,0.157895,0.124558,00:10
3,0.034462,0.025348,0.472552,0.726801,0.375276,0.481531,0.157895,0.134709,00:10
4,0.027229,0.019758,0.613505,0.760007,0.555167,0.650877,0.157895,0.141415,00:10
5,0.022019,0.025916,0.361226,0.573441,0.430242,0.443633,0.128781,0.125851,00:10
6,0.019401,0.018502,0.682405,0.782160,0.595368,0.661020,0.157895,0.201097,00:10
7,0.017196,0.020706,0.684117,0.803899,0.595553,0.593399,0.124419,0.200997,00:10
8,0.015855,0.019945,0.571257,0.738379,0.504088,0.661067,0.141373,0.299080,00:10
9,0.014451,0.015182,0.670392,0.807629,0.640062,0.693667,0.126172,0.269524,00:10


epoch,train_loss,valid_loss,foreground_acc,diceLV,diceMY,diceRV,diceEd,diceSc,time
0,0.009484,0.015336,0.700491,0.824628,0.638691,0.700053,0.117063,0.282916,00:11
1,0.009550,0.015472,0.717913,0.828310,0.638020,0.682201,0.111045,0.317108,00:11
2,0.009229,0.019167,0.575085,0.790169,0.581133,0.627399,0.119796,0.336286,00:11
3,0.009382,0.015401,0.623906,0.774423,0.635852,0.697159,0.139507,0.337458,00:11
4,0.009235,0.020016,0.658923,0.815879,0.628021,0.760816,0.157895,0.200550,00:11
5,0.008935,0.019395,0.630456,0.828486,0.629362,0.678108,0.147868,0.297261,00:11
6,0.008696,0.018575,0.686554,0.814796,0.615339,0.702497,0.166439,0.293734,00:11
7,0.008627,0.014088,0.674827,0.808393,0.649655,0.765389,0.126117,0.426100,00:11
8,0.008135,0.014073,0.673034,0.792903,0.609245,0.797862,0.140835,0.475813,00:11
9,0.007865,0.014548,0.704833,0.843025,0.638406,0.760595,0.132643,0.468116,00:11


...

epoch,train_loss,valid_loss,foreground_acc,diceFG,time
0,0.002467,0.007546,0.353998,0.386458,00:21
1,0.002373,0.009492,0.339481,0.377611,00:21
2,0.002226,0.009050,0.370707,0.387408,00:21
3,0.002180,0.009018,0.366597,0.411316,00:21
4,0.002292,0.018669,0.184635,0.272531,00:21
5,0.002123,0.009828,0.337141,0.363041,00:21
6,0.002092,0.013793,0.217345,0.290856,00:21
7,0.002160,0.007810,0.412383,0.429730,00:21
8,0.002008,0.006935,0.435928,0.427333,00:21
9,0.001860,0.011416,0.363084,0.390637,00:21


epoch,train_loss,valid_loss,foreground_acc,diceFG,time
0,0.007021,0.006806,nan,0.157904,01:12
1,0.005836,0.005776,nan,0.157895,00:20
2,0.005252,0.006450,nan,0.157904,00:20
3,0.004712,0.005217,nan,0.157981,00:20
4,0.004552,0.006037,nan,0.157895,00:20
5,0.004260,0.004886,nan,0.150376,00:20
6,0.004297,0.005248,nan,0.157895,00:20
7,0.004004,0.006618,nan,0.157895,00:20
8,0.004102,0.007112,nan,0.157895,00:20
9,0.003978,0.004740,nan,0.157895,00:20


epoch,train_loss,valid_loss,foreground_acc,diceFG,time
0,0.003411,0.004966,nan,0.152424,00:21
1,0.003340,0.004898,nan,0.143525,00:21
2,0.003262,0.004617,nan,0.153956,00:21
3,0.003193,0.005191,nan,0.178606,00:21
4,0.003191,0.004826,nan,0.171578,00:21
5,0.003080,0.005465,nan,0.177851,00:21
6,0.003096,0.005008,nan,0.153096,00:21
7,0.002973,0.006324,nan,0.199751,00:21
8,0.002846,0.004704,nan,0.201107,00:21
9,0.002803,0.004880,nan,0.179341,00:21


epoch,train_loss,valid_loss,foreground_acc,diceFG,time
0,0.007980,0.007099,nan,0.079172,00:20
1,0.005849,0.005677,nan,0.094548,00:20
2,0.005027,0.006165,nan,0.086468,00:20
3,0.004664,0.005708,nan,0.063235,00:20
4,0.004513,0.005087,nan,0.098051,00:20
5,0.004199,0.005314,nan,0.137138,00:20
6,0.004063,0.005376,nan,0.070609,00:20
7,0.004086,0.006590,nan,0.185982,00:20
8,0.003929,0.004917,nan,0.149904,00:20
9,0.003724,0.004729,nan,0.135687,00:20


epoch,train_loss,valid_loss,foreground_acc,diceFG,time
0,0.003120,0.004582,nan,0.198525,00:21
1,0.003057,0.005251,nan,0.141012,00:21
2,0.002981,0.004857,nan,0.195040,00:21
3,0.002931,0.004783,nan,0.235629,00:21
4,0.002934,0.005826,nan,0.256771,00:21
5,0.002954,0.005078,nan,0.229368,00:21
6,0.002779,0.004638,nan,0.259033,00:21
7,0.002714,0.004783,nan,0.230204,00:21
8,0.002550,0.005131,nan,0.237271,00:21
9,0.002532,0.005024,nan,0.235819,00:21
